In [1]:
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

from sklearn import preprocessing

%matplotlib inline

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [3]:
data = pd.read_csv("housepricing/HousePrices_HalfMil.csv")
data.shape

(500000, 16)

In [4]:
data.head(3)

,Area,Garage,FirePlace,Baths,White Marble,Black Marble,Indian Marble,Floors,City,Solar,Electric,Fiber,Glass Doors,Swiming Pool,Garden,Prices
0,164,2,0,2,0,1,0,0,3,1,1,1,1,0,0,43800
1,84,2,0,4,0,0,1,1,2,0,0,0,1,1,1,37550
2,190,2,4,4,1,0,0,0,2,0,0,1,0,0,0,49500


In [5]:
data.columns

Index(['Area', 'Garage', 'FirePlace', 'Baths', 'White Marble', 'Black Marble',
       'Indian Marble', 'Floors', 'City', 'Solar', 'Electric', 'Fiber',
       'Glass Doors', 'Swiming Pool', 'Garden', 'Prices'],
      dtype='object')

In [6]:
feature_cols = ['Area', 'Garage', 'FirePlace', 'Baths', 'White Marble', 'Black Marble','Indian Marble', 'Floors', 'City',
                'Solar', 'Electric', 'Fiber','Glass Doors', 'Swiming Pool', 'Garden']
cont_cols = ['Area', 'Garage', 'FirePlace', 'Baths', 'White Marble', 'Black Marble','Indian Marble', 'Floors', 'City',
             'Solar', 'Electric', 'Fiber','Glass Doors', 'Swiming Pool', 'Garden']
cat_cols = ['City']
target_col = 'Prices'

In [7]:
X = data[feature_cols]
Y = data[target_col]

In [8]:
def remove_outlier(df_in):
    for col_name in df_in.columns:
        if col_name in cont_cols:
            q1 = df_in[col_name].quantile(0.25)
            q3 = df_in[col_name].quantile(0.75)
            iqr = q3-q1 #Interquartile range
            fence_low  = q1-1.5*iqr
            fence_high = q3+1.5*iqr
            df_in[col_name] = df_in[col_name].clip(fence_low,fence_high)
        else:
            pass
    return df_in

In [9]:
X_wo = remove_outlier(X.copy())

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_wo, Y, test_size = 0.3, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(350000, 15)
(150000, 15)
(350000,)
(150000,)


In [11]:
# Scaling for KNN

scaler = preprocessing.MinMaxScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(X_train[cont_cols]), columns=cont_cols, index = X_train.index)
test_scaled = pd.DataFrame(scaler.transform(X_test[cont_cols]), columns=cont_cols, index = X_test.index)

X_train = pd.concat([train_scaled, X_train[cat_cols]], axis =1)
X_test = pd.concat([test_scaled, X_test[cat_cols]], axis =1)

In [12]:
# import the regressor 
from sklearn.tree import DecisionTreeRegressor

# create a regressor object 
regressor = DecisionTreeRegressor(random_state = 0, min_samples_leaf = 20, max_depth = 4)
  
# fit the regressor with X and Y data
regressor.fit(X_train, Y_train) 

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=20,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [13]:
from sklearn.metrics import mean_squared_error

# model evaluation for training set
y_train_predict = regressor.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

# model evaluation for testing set
y_test_predict = regressor.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

The model performance for training set
--------------------------------------
RMSE is 4518.69783501122
The model performance for testing set
--------------------------------------
RMSE is 4515.305123160281


In [14]:
df_train = pd.concat([X_train,Y_train], axis = 1)
df_train['Tree_pred'] = y_train_predict
df_train.loc[:,'Leaf_node'] = regressor.apply(X_train)

In [15]:
df_test = pd.concat([X_test,Y_test], axis = 1)
df_test['Tree_pred'] = y_test_predict
df_test.loc[:,'Leaf_node'] = regressor.apply(X_test)

In [16]:
from sklearn.neighbors import KNeighborsRegressor

df_final = pd.DataFrame()
for i in df_test['Leaf_node'].unique():
    X_train = df_train[df_train['Leaf_node']==i]
    X_test = df_test[df_test['Leaf_node']==i]
    
    if(X_train.shape[0]>10):
        # Need rule for auto selectio of neighbors
        neigh = KNeighborsRegressor(n_neighbors=5)
        neigh.fit(X_train[feature_cols],X_train[target_col])
        X_test.loc[:,'pred'] = neigh.predict(X_test[feature_cols])
    elif(X_train.shape[0]>=3):
        neigh = KNeighborsRegressor(n_neighbors=3)
        neigh.fit(X_train[feature_cols],X_train[target_col])
        X_test.loc[:,'pred'] = neigh.predict(X_test[feature_cols])        
    else:
        X_test.loc[:,'pred'] = regressor.predict(X_test[feature_cols])
        
    df_final = pd.concat([df_final,X_test], axis = 0)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
df_final.head(3)

,Area,Garage,FirePlace,Baths,White Marble,Black Marble,Indian Marble,Floors,City,Solar,Electric,Fiber,Glass Doors,Swiming Pool,Garden,City,Prices,Tree_pred,Leaf_node,pred
311150,0.395161,1.0,1.00,0.50,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,3.0,51225,51570.219165,15,50695.0
120849,0.290323,1.0,0.75,0.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,47075,51570.219165,15,47945.0
329536,0.104839,1.0,0.75,0.25,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,47425,51570.219165,15,47500.0


In [18]:
(np.sqrt(mean_squared_error(df_final[target_col], df_final['Tree_pred'])))

4515.305123160281

In [19]:
(np.sqrt(mean_squared_error(df_final[target_col], df_final['pred'])))

650.611958210627

In [20]:
# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [21]:
mean_absolute_percentage_error(df_final[target_col], df_final['Tree_pred'])

9.869007019942575

In [22]:
mean_absolute_percentage_error(df_final[target_col], df_final['pred'])

1.3363870624955665

In [23]:
df_final['Tree_pred'].std(), df_final['pred'].std()

(11246.317456360479, 12046.41591821702)

### Random Forest

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_wo, Y, test_size = 0.3, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(350000, 15)
(150000, 15)
(350000,)
(150000,)


In [25]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=3, random_state=0,
                             n_estimators=100)
regr.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [26]:
from sklearn.metrics import mean_squared_error

# model evaluation for training set
y_train_predict = regr.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

# model evaluation for testing set
y_test_predict = regr.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

print('MAPE for testing set is')
mean_absolute_percentage_error(Y_test, y_test_predict)

The model performance for training set
--------------------------------------
RMSE is 5156.604432892861
The model performance for testing set
--------------------------------------
RMSE is 5151.020262189354
MAPE for testing set is


11.297011176630543

### KNN Regression

In [30]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=5, metric="euclidean")
neigh.fit(X_train, Y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [31]:
from sklearn.metrics import mean_squared_error

# model evaluation for testing set
y_test_predict = neigh.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

print('MAPE for testing set is')
mean_absolute_percentage_error(Y_test, y_test_predict)

The model performance for testing set
--------------------------------------
RMSE is 5210.21616253427
MAPE for testing set is


11.264562968650617